<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

In [18]:
import pandas as pd
import numpy as np
import matplotlib as plt 
from random import randrange
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

## Подготовка данных

Начнем предобработку данных

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
display(data.head())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
data.shape

(10000, 14)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
data.drop(['CustomerId','RowNumber', 'Exited'], axis=1).describe().round(2)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
count,10000.00,10000.00,9091.00,10000.00,10000.00,10000.00,10000.00,10000.00
mean,650.53,38.92,5.00,76485.89,1.53,0.71,0.52,100090.24
std,96.65,10.49,2.89,62397.41,0.58,0.46,0.50,57510.49
min,350.00,18.00,0.00,0.00,1.00,0.00,0.00,11.58
25%,584.00,32.00,2.00,0.00,1.00,0.00,0.00,51002.11
50%,652.00,37.00,5.00,97198.54,1.00,1.00,1.00,100193.92
75%,718.00,44.00,7.00,127644.24,2.00,1.00,1.00,149388.25
max,850.00,92.00,10.00,250898.09,4.00,1.00,1.00,199992.48


Изучим nan значения

In [6]:
data.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

Чтобы не испортить распределения значений заполним Nan случайными значениями в промежутке от 1 до 10

In [7]:
data.loc[data['Tenure'].isna(), 'Tenure'] = data.loc[data['Tenure'].isna(), 'Tenure'].apply(lambda x: randrange(11))

In [8]:
data.duplicated().sum()

0

Уберем все неинформативные наблюдения

In [9]:
data = data.drop(['RowNumber', 'CustomerId','Surname'], axis= 1)
display(data.head())

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Закодируем методом One-hot категориальные признаки

In [10]:
data_ohe = pd.get_dummies(data)
display(data_ohe.head())

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,0,1,1,0


Теперь разобьем датасет на тренировочную, валидационную и тестовую выборки

In [11]:
target = data_ohe['Exited']
features = data_ohe.drop('Exited', axis=1)
features_train, X, target_train, y = train_test_split(features, target, test_size=0.4, stratify=target, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(X, y, test_size=0.5, stratify=y, random_state=12345)



Следом надо стандартизировать численные характеристики

In [12]:
numeric = ['Age', 'Balance', 'EstimatedSalary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
pd.options.mode.chained_assignment = None

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

## Исследование задачи

Теперь попробуем обучить модели без учета дисбаланса

In [13]:
RandomForest_model = RandomForestClassifier(random_state=12345) 
RandomForest_model.fit(features_train, target_train) 
predicted_valid = RandomForest_model.predict(features_valid)
probabilities_valid = RandomForest_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

f1 = f1_score(target_valid, predicted_valid)
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('F1 score -', f1)
print('roc auc score -', auc_roc)

F1 score - 0.5388127853881278
roc auc score - 0.8202817087890433


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [14]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

f1 = f1_score(target_valid, predicted_valid)
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('F1 score -', f1)
print('roc auc score -', auc_roc)

F1 score - 0.3088235294117647
roc auc score - 0.7874790619765495


По итогу обучения без учета дисбаланса можно сделать следующий вывод: у обеих моделей RandomForestClassifier и LogisticRegression очень низкий показатель метрики F1. Возможно уменьшение дисбаланса поможет повысить данную оценку

## Борьба с дисбалансом

In [16]:
data['Exited'].value_counts() / len(data['Exited'])

0    0.7963
1    0.2037
Name: Exited, dtype: float64

Можно заметить, что объектов класса 0 в 4 раза больше, чем объектов класса 1

Начнем разбираться с дисбалансом классов с помощью методики upsample. Попробую увеличить количество классов 1 в 4 раза

In [24]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [25]:
target_upsampled.value_counts() / len(target_upsampled)

1    0.50569
0    0.49431
Name: Exited, dtype: float64

In [46]:
for est in range(20, 50, 70):
    for depth in range(4, 6, 8):
        RandomForest_model = RandomForestClassifier(random_state=12345, n_estimators= est, max_depth= depth) 
        RandomForest_model.fit(features_upsampled, target_upsampled) 
        predicted_valid = RandomForest_model.predict(features_valid)
        probabilities_valid = RandomForest_model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]

        f1 = f1_score(target_valid, predicted_valid)
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
        if f1 > best_result:
            best_model = RandomForest_model 
            best_result = f1

print("f1 наилучшей модели на валидационной выборке:", best_result)
print('Характеристика лучшей модели:', '\n', best_model)

f1 наилучшей модели на валидационной выборке: 0.6049999999999999
Характеристика лучшей модели: 
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)


In [44]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

f1 = f1_score(target_valid, predicted_valid)
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('F1 score -', f1)
print('roc auc score -', auc_roc)

F1 score - 0.5260416666666667
roc auc score - 0.7935557074588628


Теперь, когда классы примерно одинакового объема, метрика f1 прилично повысилась у двух моделей. Особенно резко у модели логистической регрессии. 

Перейдем к следующему методу сбалансирования классов - downsampling. Сократим объем класса 0 на 40%

In [32]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.4)

In [33]:
target_downsampled.value_counts() / len(target_downsampled)

0    0.609959
1    0.390041
Name: Exited, dtype: float64

In [45]:
best_model = None
best_result = 0
for est in range(20, 50, 70):
    for depth in range(4, 6, 8):
        RandomForest_model = RandomForestClassifier(random_state=12345, n_estimators= est, max_depth= depth) 
        RandomForest_model.fit(features_downsampled, target_downsampled) 
        predicted_valid = RandomForest_model.predict(features_valid)
        probabilities_valid = RandomForest_model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        f1 = f1_score(target_valid, predicted_valid)
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
        if f1 > best_result:
            best_model = RandomForest_model 
            best_result = f1

print("f1 наилучшей модели на валидационной выборке:", best_result)
print('Характеристика лучшей модели:', '\n', best_model)

f1 наилучшей модели на валидационной выборке: 0.6049999999999999
Характеристика лучшей модели: 
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)


In [41]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight= 'balanced')
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

f1 = f1_score(target_valid, predicted_valid)
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('F1 score -', f1)
print('roc auc score -', auc_roc)

F1 score - 0.5308964316797214
roc auc score - 0.7937265986796729


Вывод: Применив методы upsampling и downsampling к данным, и, подобрав коэффициенты к этим методом, смогли достичь значения F1-меры до 0.60 на валидационных данных. При этом повысили значение метрики AUC-ROC до 0.79.

## Тестирование модели

In [49]:
RandomForest_model = RandomForestClassifier(random_state=12345, n_estimators= 50, max_depth= 8) 
RandomForest_model.fit(features_upsampled, target_upsampled) 
predicted_test = RandomForest_model.predict(features_test)
probabilities_test = best_model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
f1 = f1_score(target_test, predicted_test)
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print(f1)
print(auc_roc)

0.5910064239828694
0.8375262782042443
